In [10]:
# Import necessary Python modules
from sklearn.decomposition import PCA
from mne_icalabel import label_components
from mne.preprocessing import ICA
import copy  # This is a Python module that allows you to copy objects without changing the original object
from scipy import signal
import sklearn as sk
import matplotlib.pyplot as plt
import os
import matplotlib
import mne
import numpy as np
from scipy.stats import zscore
matplotlib.use('Agg')  # disable plotting
mne.viz.set_browser_backend('matplotlib', verbose=None)
mne.set_config('MNE_BROWSER_BACKEND', 'matplotlib')
mne.cuda.init_cuda(verbose=True)

# defining input and output directory
files_in = '../data/in/subjects/'
files_out = '../data/out/subjects/'

EEG_list = []
# loading list of subject names from txt file
names = open("./names.txt", "r")
subject_list = names.read().split('\n')
modes = ['EC', 'EO']
# Read the custom montage
montage_path = r"../data/in/MFPRL_UPDATED_V2.sfp"
montage = mne.channels.read_custom_montage(montage_path)

bad_list = [
    [['AF7'],  # 101 EC
     ['AF7']],  # 101 EO
    [[],  # 102
     ['O2']],  # 102
    [['AF7'],  # 103
     ['AF7']],  # 103
    [[],  # 104
     []],  # 104
    [['AF7'],  # 105
     ['AF7']],  # 105
    [['O1'],  # 111
     ['O1']],  # 111
    [['AFz'],  # 112
     ['AFz']],  # 112
    [['Fp1'],  # 113
     ['Fp1']],  # 113
    [[],  # 114
     []],  # 114
    [[],  # 115
     []],  # 115
    [[],  # 115
     []],  # 115
    [[],  # 117
     []],  # 117
    [['Fp1'],  # 118
     ['Fp1']],  # 118
    [[],  # 119
     []],  # 119
    [[],  # 120
     []],  # 120
    [[],  # 401
     []],  # 401
    [['T7'],  # 402
     ['T7']],  # 402
    [[],  # 403
     []],  # 403
    [[],  # 404
     []],  # 404
    [[],  # 406
     []],  # 406
    [[],  # 407
     []],  # 407
    [[],  # 408
     []],  # 408
    [['T8'],  # 409
     ['T8']],  # 409
    [[],  # 410
     []],  # 410
    [[],  # 411
     []],  # 411
    [[],  # 412
     []],  # 412
    [[],  # 413
     []],  # 413
    [[],  # 414
     []],  # 414
    [[],  # 415
     []],  # 415
    [[],  # 416
     []]  # 416
]

# Define the map of channel names using the provided keys
ch_map = {'Ch1': 'Fp1', 'Ch2': 'Fz', 'Ch3': 'F3', 'Ch4': 'F7', 'Ch5': 'LHEye', 'Ch6': 'FC5',
          # Setting FPz as GND so it matches montage
          'Ch7': 'FC1', 'Ch8': 'C3', 'Ch9': 'T7', 'Ch10': 'GND', 'Ch11': 'CP5', 'Ch12': 'CP1',
          'Ch13': 'Pz', 'Ch14': 'P3', 'Ch15': 'P7', 'Ch16': 'O1', 'Ch17': 'Oz', 'Ch18': 'O2',
          'Ch19': 'P4', 'Ch20': 'P8', 'Ch21': 'Rmastoid', 'Ch22': 'CP6', 'Ch23': 'CP2', 'Ch24': 'Cz',
          'Ch25': 'C4', 'Ch26': 'T8', 'Ch27': 'RHEye', 'Ch28': 'FC6', 'Ch29': 'FC2', 'Ch30': 'F4',
          'Ch31': 'F8', 'Ch32': 'Fp2', 'Ch33': 'AF7', 'Ch34': 'AF3', 'Ch35': 'AFz', 'Ch36': 'F1',
          'Ch37': 'F5', 'Ch38': 'FT7', 'Ch39': 'FC3', 'Ch40': 'FCz', 'Ch41': 'C1', 'Ch42': 'C5',
          'Ch43': 'TP7', 'Ch44': 'CP3', 'Ch45': 'P1', 'Ch46': 'P5', 'Ch47': 'Lneck', 'Ch48': 'PO3',
          'Ch49': 'POz', 'Ch50': 'PO4', 'Ch51': 'Rneck', 'Ch52': 'P6', 'Ch53': 'P2', 'Ch54': 'CPz',
          'Ch55': 'CP4', 'Ch56': 'TP8', 'Ch57': 'C6', 'Ch58': 'C2', 'Ch59': 'FC4', 'Ch60': 'FT8',
          'Ch61': 'F6', 'Ch62': 'F2', 'Ch63': 'AF4', 'Ch64': 'RVEye'}

for i_s, subject in enumerate(subject_list):
    for j_m, mode in enumerate(modes):

        print(subject, mode)
        if '116' not in subject:
            print('not found')
            continue
        else:
            pass
        # defining paths for current subject
        input_path = files_in+subject + '/' + mode + '/'
        output_path = files_out + subject + '/' + mode + '/'

        log_file = output_path+'log2.txt'
        log = open(log_file, "w")

        # loading in files savef from 1filter.py
        EEG = mne.io.read_raw_fif(
            output_path + subject + '_maprenamed&nfiltered.fif', preload=True)

        # MARKING BAD CHANNELS

        # TODO - get bad channels from .txt files here
        print(EEG.ch_names)
        bad_txt = bad_list[i_s][j_m]  # get list of bad channels

        picks = mne.pick_channels(EEG.ch_names, include=[], exclude=[])
        original_bads = copy.deepcopy(EEG.info["bads"])

        # This can be used to plot the data with the bad channels marked.
        # Uncomment the two lines of code below to see the plot
        # Replace 'regexp=" ."' with the tentative bad channels

        if bad_txt == [''] or len(bad_txt) == 0:
            pass
        else:
            picks = mne.pick_channels(
                EEG.ch_names, include=[], exclude=bad_txt)
            plot_obj = EEG.plot(order=picks, n_channels=len(picks))
            
        # Change list of bad channels
            original_bads = copy.deepcopy(EEG.info["bads"])
            for bad in bad_txt:
                EEG.info["bads"].append(bad)  # add a single channel
                # add a single channel to the original_bads list
                original_bads.append(bad)
            # EEG_csd.info["bads"].extend(["EEG 051", "EEG 052"])  # add a list of channels
            # original_bads["bads"].extend(["EEG 051", "EEG 052"])  # add a list of channels

            # Print the bad channels to double check
            log.write('Writing Bad Channels to Double check \n')
            log.write(str(EEG.info['bads']) + '\n')
            log.write(str(original_bads) + '\n \n \n')

            # Save the data with the bad channels marked
            # Replace with your desired output directory
            bad_channel_file = output_path + subject + '_badchannels.fif'
            EEG.save(bad_channel_file, overwrite=True)

        # ICA (Independent Component Analysis)

        # Keep a reference to the original, uncropped data
        original_EEG = EEG

        # Crop a copy of the data to three seconds for easier plotting
        cropped_EEG = EEG.copy().crop(tmin=0, tmax=3).load_data()

        # Fit average re-reference to the data
        original_EEG.set_eeg_reference('average')

        # Drop channels #10 and #21 (mastoids) before ICA
        original_EEG.drop_channels(['Rmastoid'])

        # Determine the number of PCA components
        data = original_EEG.get_data().T
        pca = PCA()
        pca.fit(data)

        # Plot the explained variance ratio
        plt.figure(figsize=(10, 5))
        plt.plot(np.cumsum(pca.explained_variance_ratio_))
        plt.xlabel('Number of Components')
        plt.ylabel('Cumulative Explained Variance')
        plt.title('Explained Variance by PCA Components')
        plt.grid(True)
        

        # Define ICA parameters
        n_components = 0.99  # Choose number of ICA components based on PCA
        ica = ICA(
            n_components=n_components,
            max_iter="auto",
            method="infomax",
            random_state=97,
            fit_params=dict(extended=True),
        )

        # Pick only EEG channels
        picks_eeg = mne.pick_types(original_EEG.info, meg=False,
                                   eeg=True, eog=False, stim=False, emg=False, exclude='bads')

        # Fit ICA using only EEG channels
        ica.fit(original_EEG, picks=picks_eeg, decim=3)

        # Plot the ICA components as time series
        ica_ts_plot = ica.plot_sources(
            original_EEG, show_scrollbars=False, show=True)
        # saving the timeseries plot
        

        # Plot the ICA components as topographies in multiple windows
        log.write("Plotting the ICA components as topographies... \n")
        n_components_actual = ica.n_components_

        # Selecting ICA components automatically using ICLabel
        ic_labels = label_components(
            original_EEG, ica, method='iclabel')
        component_labels = ic_labels["labels"]  # Extract the labels
        # Extract the probabilities
        component_probabilities = ic_labels["y_pred_proba"]


        # ICLabel scores
        EEG_list.append(original_EEG)
        print("Initializing labels file \n")
        label_file = output_path+'labels.txt'
        labelf = open(label_file, "r")
        prob_file = output_path+'probs.txt'
        probf = open(prob_file, "r")

        labels = labelf.read().split('\n')
        

        # Exclude components based on label and probability
        exclude_idx = [idx for idx, (label, prob) in enumerate(zip(labels, component_probabilities))
               if label.strip() not in ["brain", "other"] or prob < 0.70]
        
        # Copy the original interpolated EEG data
        reconst_EEG = original_EEG.copy()

        # Apply the ICA transformation, excluding certain components
        ica.apply(reconst_EEG, exclude=exclude_idx)

        # Plot the original data and set the window title
        fig = original_EEG.plot(show_scrollbars=False)

        # Plot the reconstructed data and set the window title
        fig = reconst_EEG.plot(show_scrollbars=False)

        # Save the preprocessed data
        output_dir = r'../data/out'
        preprocessed_file = output_path + subject+'_ICA.fif'
        reconst_EEG.save(preprocessed_file, overwrite=True)

        #################################################################################

        # Interpolate bad channels
        
        # Replace NaN or inf values in channel locations with zero
        new_chs = original_EEG.info['chs'].copy()
        for ch in new_chs:
            ch['loc'] = np.nan_to_num(ch['loc'], nan=0.0, posinf=0.0, neginf=0.0)

        new_info = mne.create_info(
            [ch['ch_name'] for ch in new_chs], original_EEG.info['sfreq'], ch_types='eeg')
        original_EEG = mne.io.RawArray(original_EEG.get_data(), new_info)
        original_EEG.set_montage(mne.channels.make_dig_montage(
            ch_pos={ch['ch_name']: ch['loc'][:3] for ch in new_chs}))
        # Set the bad channels back to the original list
        original_EEG.info['bads'] = original_bads

        # Repeat for cropped_EEG
        new_chs = cropped_EEG.info['chs'].copy()
        for ch in new_chs:
            ch['loc'] = np.nan_to_num(ch['loc'], nan=0.0, posinf=0.0, neginf=0.0)

        new_info = mne.create_info(
            [ch['ch_name'] for ch in new_chs], cropped_EEG.info['sfreq'], ch_types='eeg')
        cropped_EEG = mne.io.RawArray(cropped_EEG.get_data(), new_info)
        cropped_EEG.set_montage(mne.channels.make_dig_montage(
            ch_pos={ch['ch_name']: ch['loc'][:3] for ch in new_chs}))
        # Set the bad channels back to the original list
        cropped_EEG.info['bads'] = original_bads

        # Pick types and interpolate bads
        original_EEG_data = original_EEG.copy().pick_types(
            meg=False, eeg=True, exclude=[])
        original_EEG_data_interp = original_EEG_data.copy().interpolate_bads(reset_bads=False)

        cropped_EEG_data = cropped_EEG.copy().pick_types(meg=False, eeg=True, exclude=[])
        cropped_EEG_data_interp = cropped_EEG_data.copy().interpolate_bads(reset_bads=False)

        # Plot the data before and after interpolation
        for title, data in zip(["cropped orig.", "cropped interp."], [cropped_EEG_data, cropped_EEG_data_interp]):
            with mne.viz.use_browser_backend("matplotlib"):
                fig = data.plot(butterfly=True, color="#00000022", bad_color="r")
            fig.subplots_adjust(top=0.9)
            fig.suptitle(title, size="xx-large", weight="bold")

        # Save the interpolated data
        output_dir = r'../data/out'
        preprocessed_file = output_path+ subject + '_interpolated.fif'
        original_EEG_data_interp.save(preprocessed_file, overwrite=True)

        #################################################################################

        # EPOCHING

        # Define epoch parameters
        name = subject + '_eventchan'  # --> change for each condition
        # Latency rate/Sampling rate
        epoch_no = np.floor(reconst_EEG.get_data(
        ).shape[1] / reconst_EEG.info['sfreq'])

        # Create a list of onset times for your events
        onsets = np.arange(0, reconst_EEG.get_data(
        ).shape[1] / reconst_EEG.info['sfreq'], 1)

        # Create a list of event durations (all zeros if the events are instantaneous)
        durations = np.zeros_like(onsets)

        # Create a list of event descriptions
        descriptions = ['Event'] * len(onsets)

        # Create an Annotations object
        annotations = mne.Annotations(onsets, durations, descriptions)

        # Add the annotations to the Raw object
        reconst_EEG.set_annotations(annotations)

        # Now you can extract the events from the annotations
        events, event_id = mne.events_from_annotations(reconst_EEG)

        # Define epoching parameters
        name = subject + '_epoch'  # --> change the name of condition
        codes = ['1']
        tmin = -0.5  # Start of the epoch (in seconds)
        tmax = 0.5  # End of the epoch (in seconds)

        # Create epochs without rejection to keep all data
        epochs_all = mne.Epochs(reconst_EEG, events, event_id=event_id,
                                tmin=tmin, tmax=tmax, proj=True, baseline=None, preload=True)

        # Apply z-score normalization and keep track of which epochs exceed the threshold
        zscore_threshold = 6
        to_drop = []

        temp_data = np.zeros_like(epochs_all._data)

        for i in range(len(epochs_all)):
            temp_data[i] = zscore(epochs_all._data[i], axis=1)
            if np.any(np.abs(temp_data[i]) > zscore_threshold):
                to_drop.append(i)

        # Now we can drop the epochs that exceeded the threshold
        epochs_all.drop(to_drop)

        # Resample and decimate the epochs
        current_sfreq = epochs_all.info['sfreq']
        # Hz chaging this according to  https://doi.org/10.1046/j.1440-1819.2000.00729.x
        desired_sfreq = 512

        # Apply the resampling
        epochs_all.resample(desired_sfreq, npad='auto')

        # Get the data from all epochs
        data_all = epochs_all.get_data()

        # Plot the data
        epochs_all.plot()

        # Save the filtered data
        # Replace with your desired output directory
        output_dir = r'../data/out'
        preprocessed_file = output_path + subject + '_epoched.fif'
        epochs_all.save(preprocessed_file, overwrite=True)


     

101 EC
not found
101 EO
not found
102 EC
not found
102 EO
not found
103 EC
not found
103 EO
not found
104 EC
not found
104 EO
not found
105 EC
not found
105 EO
not found
111 EC
not found
111 EO
not found
112 EC
not found
112 EO
not found
113 EC
not found
113 EO
not found
114 EC
not found
114 EO
not found
115 EC
not found
115 EO
not found
116 EC
Opening raw data file ../data/out/subjects/116/EC/116_maprenamed&nfiltered.fif...
    Range : 0 ... 116599 =      0.000 ...   116.599 secs
Ready.
Reading 0 ... 116599  =      0.000 ...   116.599 secs...
['Fp1', 'Fz', 'F3', 'F7', 'LHEye', 'FC5', 'FC1', 'C3', 'T7', 'GND', 'CP5', 'CP1', 'Pz', 'P3', 'P7', 'O1', 'Oz', 'O2', 'P4', 'P8', 'CP6', 'CP2', 'Cz', 'C4', 'T8', 'RHEye', 'FC6', 'FC2', 'F4', 'F8', 'Fp2', 'AF7', 'AF3', 'AFz', 'F1', 'F5', 'FT7', 'FC3', 'FCz', 'C1', 'C5', 'TP7', 'CP3', 'P1', 'P5', 'Lneck', 'PO3', 'POz', 'PO4', 'Rneck', 'P6', 'P2', 'CPz', 'CP4', 'TP8', 'C6', 'C2', 'FC4', 'FT8', 'F6', 'F2', 'AF4', 'RVEye', 'Rmastoid']
EEG channel type

/tmp/ipykernel_3776169/1332438492.py:17: RuntimeWarning: module cupy not found, CUDA not enabled
  mne.cuda.init_cuda(verbose=True)
/tmp/ipykernel_3776169/1332438492.py:126: RuntimeWarning: This filename (../data/out/subjects/116/EC/116_maprenamed&nfiltered.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  EEG = mne.io.read_raw_fif(


Fitting ICA to data using 58 channels (please be patient, this may take a while)
Selecting by explained variance: 10 components
Computing Extended Infomax ICA
Fitting ICA took 9.3s.
Creating RawArray with float64 data, n_channels=13, n_times=116600
    Range : 0 ... 116599 =      0.000 ...   116.599 secs
Ready.
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


/tmp/ipykernel_3776169/1332438492.py:224: RuntimeWarning: The provided Raw instance is not filtered between 1 and 100 Hz. ICLabel was designed to classify features extracted from an EEG dataset bandpass filtered between 1 and 100 Hz (see the 'filter()' method for Raw and Epochs instances).
  ic_labels = label_components(


Initializing labels file 

Applying ICA to Raw instance
    Transforming to ICA space (10 components)
    Zeroing out 1 ICA component
    Projecting back using 58 PCA components
Overwriting existing file.
Writing /home/prakhar7/mfprl/sample/neuroconn/batch/../data/out/subjects/116/EC/116_ICA.fif
Closing /home/prakhar7/mfprl/sample/neuroconn/batch/../data/out/subjects/116/EC/116_ICA.fif
[done]
Creating RawArray with float64 data, n_channels=63, n_times=116600
    Range : 0 ... 116599 =      0.000 ...   116.599 secs
Ready.
Creating RawArray with float64 data, n_channels=64, n_times=3001
    Range : 0 ... 3000 =      0.000 ...     3.000 secs
Ready.
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


...

/tmp/ipykernel_3776169/1332438492.py:311: RuntimeWarning: This filename (/home/prakhar7/mfprl/sample/neuroconn/batch/../data/out/subjects/116/EO/116_interpolated.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  original_EEG_data_interp.save(preprocessed_file, overwrite=True)


Dropped 61 epochs: 0, 1, 4, 5, 10, 13, 18, 20, 22, 23, 24, 26, 27, 30, 31, 33, 34, 35, 37, 38, 41, 42, 43, 46, 49, 50, 51, 52, 54, 55, 56, 60, 61, 62, 64, 65, 66, 67, 68, 70, 71, 72, 74, 75, 77, 82, 83, 85, 86, 87, 88, 90, 93, 96, 97, 98, 99, 103, 108, 114, 115


/tmp/ipykernel_3776169/1332438492.py:378: FutureWarning: The current default events=None is deprecated and will change to events=True in MNE 1.6. Set events=False to suppress this warning.
  epochs_all.plot()


117 EC
not found
117 EO
not found
118 EC
not found
118 EO
not found
119 EC
not found
119 EO
not found
120 EC
not found
120 EO
not found
401 EC
not found
401 EO
not found
402 EC
not found
402 EO
not found
403 EC
not found
403 EO
not found
404 EC
not found
404 EO
not found
406 EC
not found
406 EO
not found
407 EC
not found
407 EO
not found
408 EC
not found
408 EO
not found
409 EC
not found
409 EO
not found
410 EC
not found
410 EO
not found
411 EC
not found
411 EO
not found
412 EC
not found
412 EO
not found
413 EC
not found
413 EO
not found
414 EC
not found
414 EO
not found
415 EC
not found
415 EO
not found
416 EC
not found
416 EO
not found


/tmp/ipykernel_3776169/1332438492.py:384: RuntimeWarning: This filename (../data/out/subjects/116/EO/116_epoched.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs_all.save(preprocessed_file, overwrite=True)


In [1]:
#graph code

In [2]:
import numpy as np
from scipy.signal import hilbert
from scipy.stats import pearsonr
import networkx as nx

def compute_amplitude_coupling(data, labels):
    """
    Compute the amplitude coupling between all pairs of regions and extract additional information.

    Parameters:
    data - time series data for all regions (epochs x regions x time)
    labels - list of labels representing regions

    Returns:
    A dictionary with amplitude coupling values and additional information.
    """
    n_regions = data.shape[1]
    coupling_info = {}

    # Compute the envelope of the analytic signal for each region
    envelopes = np.abs(hilbert(data, axis=2))
    mean_envelopes = envelopes.mean(axis=0)
    signs = np.sign(mean_envelopes)

    for i in range(n_regions):
        for j in range(i + 1, n_regions):
            corr, _ = pearsonr(envelopes[:, i].ravel(), envelopes[:, j].ravel())

            # Standardize the correlation to a [0, 1] scale with 0.5 as no connectivity, <0.5 as negative connectivity, and >0.5 as positive connectivity
            standardized_corr = (corr + 1) / 2  # This shifts the [-1, 1] range to [0, 1]

            # Determine the nature of coupling based on the correlation and mean envelopes' signs
            if corr > 0:
                if signs[i] == signs[j] == 1:
                    nature_of_coupling = 'co-activation'
                elif signs[i] == signs[j] == -1:
                    nature_of_coupling = 'co-deactivation'
                else:
                    nature_of_coupling = 'complex-coupling'  # Different or zero signs, positive correlation
            else:
                nature_of_coupling = 'anti-correlation' if signs[i] != signs[
                    j] else 'complex-coupling'  # Same signs, negative correlation

            # Record the coupling information
            coupling_info[(labels[i].name, labels[j].name)] = {
                'correlation': corr,  # Original correlation value
                'standardized_correlation': standardized_corr,  # Standardized correlation value
                'nature_of_coupling': nature_of_coupling,
                'activation_magnitudes': (mean_envelopes[i], mean_envelopes[j])
            }

    return coupling_info


# Function to perform aggregated bootstrapping and find optimal alpha and upper threshold
    """
    Perform aggregated bootstrapping and determine optimal alpha and upper threshold for a set of windowed graphs.

    Parameters:
    - windowed_graphs (list): A list of networkx graphs representing data in consecutive windows.
    - num_iterations (int): Number of iterations for bootstrapping the aggregated edge weights (default is 1000).
    - percentile (int): Desired percentile for determining the upper threshold (default is 95).
    - alpha_start (float): Starting value for the range of alpha values to be tested (default is 0.001).
    - alpha_end (float): Ending value for the range of alpha values to be tested (default is 0.1).
    - num_alphas (int): Number of alpha values to be tested within the specified range (default is 100).

    Returns:
    - optimal_alpha (float): The optimal alpha value determined for the aggregated data.
    - upper_threshold (float): The upper threshold calculated using bootstrapping.
    """
def aggregated_bootstrapping_and_threshold(windowed_graphs, num_iterations=1000, percentile=95, alpha_start=0.001,
                                           alpha_end=0.1, num_alphas=100):
    # Aggregate edge weights from all windowed graphs
    all_edge_weights = np.concatenate(
        [np.array([data['weight'] for _, _, data in G.edges(data=True)]) for G in windowed_graphs])

    # Perform bootstrapping on aggregated edge weights
    bootstrap_weights = []
    for _ in range(num_iterations):
        random_weights = np.random.choice(all_edge_weights, size=len(all_edge_weights), replace=True)
        bootstrap_weights.extend(random_weights)

    # Determine upper threshold for aggregated data
    upper_threshold = np.percentile(bootstrap_weights, percentile)

    # Test range of alphas to determine optimal alpha for aggregated data
    alphas = np.linspace(alpha_start, alpha_end, num_alphas)
    avg_connectivities = []
    for alpha in alphas:
        connectivities = []
        for G in windowed_graphs:
            G_filtered = G.copy()
            for u, v, weight in G.edges(data='weight'):
                if weight > upper_threshold or (G_filtered[u][v]['weight'] ** 2 / sum(
                        [d['weight'] ** 2 for _, _, d in G_filtered.edges(u, data=True)]) < alpha):
                    G_filtered.remove_edge(u, v)
            connectivities.append(np.mean(
                nx.convert_matrix.to_numpy_array(G_filtered)[np.nonzero(nx.convert_matrix.to_numpy_array(G_filtered))]))
        avg_connectivities.append(np.mean(connectivities))

    optimal_alpha_idx = np.argmin(np.abs(np.diff(avg_connectivities)))
    return alphas[optimal_alpha_idx], upper_threshold


# Function to apply aggregated threshold and disparity filter to a graph
def apply_aggregated_filter(G, optimal_alpha, upper_threshold):
    G_filtered = G.copy()
    for u, v, data in G.edges(data=True):
        if data['weight'] > upper_threshold:
            G_filtered.remove_edge(u, v)

        elif data['weight'] ** 2 / sum(
                [d['weight'] ** 2 for _, _, d in G_filtered.edges(u, data=True)]) < optimal_alpha:
            G_filtered.remove_edge(u, v)

    return G_filtered

def compute_cross_correlation(data_window):
    """Compute cross-correlation for given data window."""
    # Reshape the data to be 2D

    data_2D = data_window.reshape(data_window.shape[0], -1)
    correlation_matrix = np.corrcoef(data_2D, rowvar=True)
    return correlation_matrix

    # Compute dPLI at the level of regions


def compute_dPLI(data):
    print('Computing dPLI')
    n_regions = data.shape[1]  # Compute for regions
    dPLI_matrix = np.zeros((n_regions, n_regions))
    print(data)
    analytic_signal = hilbert(data)
    phase_data = np.angle(analytic_signal)
    for i in range(n_regions):
        for j in range(n_regions):
            if i != j:
                phase_diff = phase_data[:, i] - phase_data[:, j]
                dPLI_matrix[i, j] = np.abs(
                    np.mean(np.exp(complex(0, 1) * phase_diff)))
    return dPLI_matrix

# dPLI_matrix = compute_dPLI(label_time_courses) --> computing static, fc for the entire dataset


def disparity_filter(G, alpha=0.01):
    disparities = {}
    for i, j, data in G.edges(data=True):
        weight_sum_square = sum(
            [d['weight']**2 for _, _, d in G.edges(i, data=True)])
        disparities[(i, j)] = data['weight']**2 / weight_sum_square

    G_filtered = G.copy()
    for (i, j), disparity in disparities.items():
        if disparity < alpha:
            G_filtered.remove_edge(i, j)
    return G_filtered


def graph_to_matrix(graph, size):
    matrix = np.zeros((size, size))
    for i, j, data in graph.edges(data=True):
        matrix[i, j] = data['weight']
        matrix[j, i] = data['weight']  # Ensure symmetry
    return matrix


def threshold_matrix(matrix):
    G_temp = nx.convert_matrix.from_numpy_array(matrix)
    G_temp_thresholded = disparity_filter(G_temp)

    matrix_thresholded = np.zeros_like(matrix)
    for i, j, data in G_temp_thresholded.edges(data=True):
        matrix_thresholded[i, j] = data['weight']
        matrix_thresholded[j, i] = data['weight']
    return matrix_thresholded


def threshold_graph_by_density(G, density=0.1, directed=False):
    if density < 0 or density > 1:
        raise ValueError("Density value must be between 0 and 1.")
    num_edges_desired = int(G.number_of_edges() * density)
    sorted_edges = sorted(G.edges(data=True), key=lambda x: x[2]['weight'],
                          reverse=True)
    if directed:
        G_thresholded = nx.DiGraph()
    else:
        G_thresholded = nx.Graph()
    G_thresholded.add_edges_from(sorted_edges[:num_edges_desired])
    return G_thresholded

# Convert dPLI to PLI


def dpli_to_pli(dpli_matrix):
    return 2 * np.abs(dpli_matrix - 0.5)


def compute_disparity(G):
    """
    Compute the disparity Y(i,j) for each edge in the graph.
    """
    disparities = {}
    for i, j, data in G.edges(data=True):
        weight_sum_square = sum(
            [d['weight']**2 for _, _, d in G.edges(i, data=True)])
        disparities[(i, j)] = data['weight']**2 / weight_sum_square
    return disparities

# Compute wPLI at the level of regions
def compute_wPLI(data):
    n_regions = data.shape[1]
    wPLI_matrix = np.zeros((n_regions, n_regions))

    # Compute the phase of the analytic signal
    analytic_signal = hilbert(data)
    phase_data = np.angle(analytic_signal)

    for i in range(n_regions):
        for j in range(i+1, n_regions):  # Only compute for upper triangle
            phase_diff = phase_data[i] - phase_data[j]
            imag_part = np.abs(np.imag(np.exp(1j * phase_diff)))
            wPLI_matrix[i, j] = np.mean(imag_part) / np.mean(np.abs(np.exp(1j * phase_diff)))
            wPLI_matrix[j, i] = wPLI_matrix[i, j]  # Symmetric matrix

    return wPLI_matrix

In [4]:
# This script is used to build the a two-step thresholding procedure (bootstrapping + disparity filter)
# The output should be a list of thresholded dPLI matrices for each window per participant
from sklearn.decomposition import PCA

from scipy import signal
import sklearn as sk
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
from scipy.stats import zscore


# defining input and output directory
files_in = '../data/in/subjects/'
files_out = '../data/out/subjects/'


# loading list of subject names from txt file
names = open("./names.txt", "r")
subject_list = names.read().split('\n')
modes = ['EC', 'EO']


all_graphs = []

for subject in subject_list[:2]:
    for mode in modes:
        tc_file = files_out + subject +'/'+mode +'/'+subject+'_label_time_courses.npy'
        label_time_courses = np.load(tc_file)
        sampling_rate = 512  # in Hz
        window_length_seconds = 1  # desired window length in seconds
        step_size_seconds = 0.5  # desired step size in seconds

        # Convert time to samples
        window_length_samples = int(window_length_seconds * sampling_rate)
        step_size_samples = int(step_size_seconds * sampling_rate)

        # Calculate total duration in samples
        num_epochs_per_hemisphere = label_time_courses.shape[0] / 2
        duration_per_epoch = label_time_courses.shape[2] / sampling_rate
        total_duration_samples = int(num_epochs_per_hemisphere * duration_per_epoch * sampling_rate)

        # Compute dPLI for each window
        num_windows = int((total_duration_samples - window_length_samples) / step_size_samples) + 1
        windowed_dpli_matrices = []

        for win_idx in range(num_windows):
            start_sample = win_idx * step_size_samples
            end_sample = start_sample + window_length_samples

            if end_sample > total_duration_samples:
                break

            start_epoch = start_sample // label_time_courses.shape[2]
            start_sample_in_epoch = start_sample % label_time_courses.shape[2]
            end_epoch = end_sample // label_time_courses.shape[2]
            end_sample_in_epoch = end_sample % label_time_courses.shape[2]

            if start_epoch == end_epoch:
                windowed_data = label_time_courses[start_epoch, :, start_sample_in_epoch:end_sample_in_epoch]
            else:
                first_part = label_time_courses[start_epoch, :, start_sample_in_epoch:]
                samples_needed_from_second_epoch = window_length_samples - first_part.shape[1]
                second_part = label_time_courses[end_epoch, :, :samples_needed_from_second_epoch]
                windowed_data = np.concatenate((first_part, second_part), axis=1)

            dpli_result = compute_dPLI(windowed_data)
            windowed_dpli_matrices.append(dpli_result)
            all_graphs.append(dpli_result)

        all_graphs.append(windowed_dpli_matrices)

print(all_graphs)

alpha, thresh = aggregated_bootstrapping_and_threshold(all_graphs)

print(alpha, thresh)











#     # Convert each windowed dPLI matrix to a graph
#         windowed_graphs = [nx.convert_matrix.from_numpy_array(matrix, create_using=nx.DiGraph) for matrix in windowed_dpli_matrices]

#         # Perform aggregated bootstrapping and find optimal alpha and upper threshold
#         optimal_alpha, upper_threshold = aggregated_bootstrapping_and_threshold(windowed_graphs, num_iterations=1000, percentile=95)

#         # Apply the aggregated filter to each windowed graph
#         thresholded_dpli_matrices = []
#         for G_dPLI in windowed_graphs:
#             G_dPLI_thresholded = apply_aggregated_filter(G_dPLI, optimal_alpha, upper_threshold)
#             dpli_matrix_thresholded = nx.convert_matrix.to_numpy_array(G_dPLI_thresholded)
#             thresholded_dpli_matrices.append(dpli_matrix_thresholded)

#         # thresholded_dpli_matrices contains the thresholded dPLI matrices for each window

Computing dPLI
[[1.83680585e-14 2.01720405e-14 2.14687616e-14 ... 2.04087574e-14
  2.19071231e-14 2.21807927e-14]
 [7.04400985e-15 7.62371581e-15 8.32286019e-15 ... 9.01718750e-15
  1.01864648e-14 1.08103766e-14]
 [2.45988244e-14 2.58460618e-14 2.68820949e-14 ... 2.63200496e-14
  2.65423517e-14 2.63217674e-14]
 ...
 [3.09118982e-14 3.63545728e-14 4.17833933e-14 ... 3.75112945e-14
  4.43660577e-14 4.88094130e-14]
 [2.58782203e-14 2.89943579e-14 3.14168247e-14 ... 2.94401242e-14
  3.10342775e-14 3.14452376e-14]
 [2.24553720e-14 2.49703178e-14 2.70731551e-14 ... 2.60019396e-14
  2.80232888e-14 2.85783936e-14]]
Computing dPLI
[[2.93152987e-14 2.76088255e-14 2.62841998e-14 ... 1.07621330e-14
  1.01828073e-14 9.45355600e-15]
 [1.02363610e-14 9.40552415e-15 8.74764721e-15 ... 7.56581687e-15
  7.29129010e-15 6.80209215e-15]
 [3.08508373e-14 2.84170169e-14 2.69615432e-14 ... 3.03048602e-14
  2.94020044e-14 2.88641961e-14]
 ...
 [4.59511613e-14 4.09111696e-14 3.78083049e-14 ... 1.40724089e-14
  

Computing dPLI
[[2.6715135e-14 2.3958449e-14 2.0683249e-14 ... 2.8231840e-14
  2.8469458e-14 2.9798304e-14]
 [9.5184751e-15 8.9300075e-15 8.2079102e-15 ... 1.0134373e-14
  1.1316479e-14 1.2768545e-14]
 [2.8744080e-14 2.7699534e-14 2.7153064e-14 ... 2.2476622e-14
  2.0781206e-14 2.0284225e-14]
 ...
 [3.6526632e-14 3.1401151e-14 2.6469053e-14 ... 1.9271131e-14
  1.8058011e-14 1.9107124e-14]
 [2.9349322e-14 2.6886198e-14 2.4268959e-14 ... 3.3122078e-14
  3.2807456e-14 3.3439434e-14]
 [2.4848586e-14 2.3007478e-14 2.1154206e-14 ... 3.0960399e-14
  3.0575199e-14 3.0825091e-14]]
Computing dPLI
[[1.5433139e-14 1.3580610e-14 1.1417165e-14 ... 1.5667057e-14
  1.5879159e-14 1.7800745e-14]
 [7.5879769e-15 7.2038145e-15 6.7935498e-15 ... 6.1501038e-15
  6.3492167e-15 7.1063032e-15]
 [3.1826843e-14 3.0411068e-14 2.8444610e-14 ... 2.2894448e-14
  2.1988338e-14 2.2832256e-14]
 ...
 [3.3171203e-14 2.9264544e-14 2.6368459e-14 ... 1.7706558e-14
  1.8924822e-14 2.2489324e-14]
 [2.3856561e-14 2.1659815e-14

Computing dPLI
[[1.1628547e-14 1.0126009e-14 8.5114428e-15 ... 3.1043808e-14
  3.0203379e-14 2.8798663e-14]
 [5.4693078e-15 4.6072799e-15 4.2398065e-15 ... 1.3656116e-14
  1.3795915e-14 1.3583606e-14]
 [1.4477978e-14 1.5428842e-14 1.6725662e-14 ... 3.8707142e-14
  3.9925454e-14 4.1018092e-14]
 ...
 [1.8499266e-14 1.7134194e-14 1.6784925e-14 ... 6.1712618e-14
  6.2682742e-14 6.0798365e-14]
 [1.8712835e-14 1.7715071e-14 1.6726648e-14 ... 4.2164328e-14
  4.2334795e-14 4.1662566e-14]
 [1.7249990e-14 1.6259987e-14 1.5104568e-14 ... 3.6458910e-14
  3.6449488e-14 3.5813237e-14]]
Computing dPLI
[[1.64826445e-14 1.50266186e-14 1.49812718e-14 ... 5.73519444e-15
  5.25040487e-15 4.98171544e-15]
 [6.87684069e-15 7.40652894e-15 8.16393648e-15 ... 4.67622580e-15
  4.59705236e-15 4.64911651e-15]
 [2.64055355e-14 2.59869048e-14 2.57500286e-14 ... 1.75669551e-14
  1.77279812e-14 1.81267151e-14]
 ...
 [3.25395059e-14 2.88801270e-14 2.62294831e-14 ... 1.54650564e-14
  1.36216129e-14 1.25575726e-14]
 [2.7

Computing dPLI
[[4.17427052e-14 4.16272851e-14 4.11218741e-14 ... 2.05012534e-14
  1.89803329e-14 1.72832363e-14]
 [1.35801548e-14 1.28257864e-14 1.25423938e-14 ... 1.06784834e-14
  9.56790968e-15 8.82529218e-15]
 [4.53473894e-14 4.48178211e-14 4.41402930e-14 ... 3.22984403e-14
  3.13873378e-14 3.02574738e-14]
 ...
 [6.19272293e-14 6.45312525e-14 6.63968663e-14 ... 4.69590356e-14
  4.40084540e-14 3.93023931e-14]
 [5.33485677e-14 5.36590019e-14 5.31223523e-14 ... 2.72316857e-14
  2.58752844e-14 2.41305660e-14]
 [4.67949586e-14 4.80133748e-14 4.85934636e-14 ... 2.34722554e-14
  2.19670524e-14 2.02260677e-14]]
Computing dPLI
[[2.5818305e-14 2.5507311e-14 2.5059598e-14 ... 3.4756327e-14
  3.4959933e-14 3.4234958e-14]
 [1.2288528e-14 1.1657619e-14 1.1211633e-14 ... 1.2433992e-14
  1.2562543e-14 1.2357972e-14]
 [3.6714216e-14 3.5788829e-14 3.5443873e-14 ... 3.3071504e-14
  3.4067519e-14 3.4224258e-14]
 ...
 [4.7714769e-14 4.8858693e-14 4.9957288e-14 ... 5.0821093e-14
  5.3949165e-14 5.488997

Computing dPLI
[[2.1076302e-14 2.0910399e-14 2.0022856e-14 ... 2.1595188e-14
  2.1495228e-14 2.2634071e-14]
 [8.3408969e-15 9.7432912e-15 1.1027799e-14 ... 1.0420986e-14
  1.0544225e-14 1.0823075e-14]
 [1.9219172e-14 2.0430772e-14 2.2168318e-14 ... 2.9610639e-14
  3.1852996e-14 3.2603897e-14]
 ...
 [3.1199093e-14 3.1990181e-14 3.3426858e-14 ... 6.0532972e-14
  5.9454564e-14 5.4284509e-14]
 [2.8149348e-14 2.8112824e-14 2.7652675e-14 ... 2.8526264e-14
  2.9022483e-14 2.9650924e-14]
 [2.7521568e-14 2.6587726e-14 2.5347430e-14 ... 2.7271850e-14
  2.8574653e-14 2.9034355e-14]]
Computing dPLI
[[2.34970006e-14 2.68544291e-14 2.96155990e-14 ... 3.26232978e-14
  3.20567039e-14 3.00713332e-14]
 [7.88769108e-15 9.24003594e-15 1.10397607e-14 ... 1.09577806e-14
  1.09500947e-14 1.03873285e-14]
 [2.60169016e-14 2.81209382e-14 3.06904567e-14 ... 2.84697057e-14
  3.02535029e-14 3.13796637e-14]
 ...
 [1.75037139e-14 2.15490349e-14 2.74565052e-14 ... 4.06261667e-14
  4.03203370e-14 3.71613479e-14]
 [2.6

...

Computing dPLI
[[1.92326263e-15 1.48474775e-15 1.23866583e-15 ... 6.23282376e-15
  6.01536668e-15 5.90457985e-15]
 [6.02076567e-15 5.88779759e-15 5.72984204e-15 ... 1.32176400e-15
  7.17472045e-17 4.51205345e-16]
 [1.59023321e-14 1.71964714e-14 1.61519578e-14 ... 1.57132337e-14
  1.29800015e-14 9.60559333e-15]
 ...
 [1.38597638e-14 1.64584261e-14 1.86958230e-14 ... 1.22657139e-14
  1.70149183e-14 1.89259297e-14]
 [7.11436694e-15 8.25075566e-15 9.62203594e-15 ... 1.18941840e-14
  1.21267471e-14 1.24796608e-14]
 [4.85254333e-15 5.42682066e-15 6.18889324e-15 ... 1.10212048e-14
  1.12046323e-14 1.12305066e-14]]
Computing dPLI
[[ 3.72026484e-15  4.89019988e-15  5.74704824e-15 ...  6.74249237e-15
   7.31921170e-15  7.35762803e-15]
 [ 1.02783994e-16 -1.24993094e-15 -1.56934093e-15 ...  9.38972953e-15
   1.10860621e-14  1.19018810e-14]
 [ 4.88280613e-15  3.95627363e-15  3.47238616e-15 ...  9.98392314e-15
   9.46172140e-15  1.05873469e-14]
 ...
 [ 1.56964641e-14  1.05667411e-14  6.20786847e-15 

Computing dPLI
[[5.14383288e-15 4.63509558e-15 4.16866164e-15 ... 2.91642253e-15
  2.53383490e-15 2.46835015e-15]
 [1.91410875e-15 1.29574389e-15 6.33868265e-16 ... 5.17312498e-15
  5.90834110e-15 5.90015919e-15]
 [1.46039559e-14 1.59108736e-14 1.60517097e-14 ... 1.86140013e-14
  1.66668843e-14 1.37067930e-14]
 ...
 [7.14146353e-15 8.71417247e-15 9.77490251e-15 ... 6.43603543e-15
  1.09685473e-14 1.47853463e-14]
 [1.09403512e-14 1.16256305e-14 1.24793745e-14 ... 9.99237229e-15
  9.54983485e-15 9.10355098e-15]
 [9.82333501e-15 1.03271409e-14 1.06556770e-14 ... 6.32786932e-15
  6.23915490e-15 6.11847600e-15]]
Computing dPLI
[[ 7.2659054e-15  6.7141350e-15  5.9474105e-15 ...  4.1967162e-15
   4.2713940e-15  4.0351129e-15]
 [ 6.5659229e-15  5.3874171e-15  4.2577441e-15 ... -1.2080430e-16
  -1.1504323e-16  4.9683565e-17]
 [ 9.7077048e-15  9.8649785e-15  1.1632206e-14 ...  1.2547604e-14
   1.4893385e-14  1.5778320e-14]
 ...
 [ 1.2730305e-14  1.5227381e-14  1.6223300e-14 ...  2.0155860e-14
  

Computing dPLI
[[2.75267200e-15 3.46792229e-15 3.83990782e-15 ... 1.31992308e-15
  1.27148952e-15 2.21409288e-15]
 [4.84771101e-15 4.97459612e-15 5.38787027e-15 ... 2.15355205e-15
  2.85853439e-15 3.22333115e-15]
 [6.43213823e-15 4.80234435e-15 4.48095210e-15 ... 4.05930757e-15
  5.40364711e-15 8.97471557e-15]
 ...
 [1.25623380e-14 1.18965193e-14 1.11520637e-14 ... 1.01193425e-14
  1.09650744e-14 1.24925544e-14]
 [9.97176313e-15 1.06403373e-14 1.10802591e-14 ... 8.24144762e-15
  8.53503601e-15 9.21256073e-15]
 [7.02443026e-15 7.15580125e-15 7.13923160e-15 ... 5.46458393e-15
  5.67715236e-15 6.22085942e-15]]
Computing dPLI
[[3.32440421e-15 3.34205595e-15 3.03178439e-15 ... 4.32296648e-15
  5.33213932e-15 6.53686284e-15]
 [3.35802358e-15 5.77672658e-15 8.07823284e-15 ... 4.86729780e-15
  3.80957303e-15 2.24174618e-15]
 [9.55831280e-15 1.32424165e-14 1.48374185e-14 ... 1.05916490e-14
  1.15545662e-14 1.35034348e-14]
 ...
 [1.90767236e-14 2.76140991e-14 3.20757689e-14 ... 1.55104675e-14
  

Computing dPLI
[[3.8406646e-15 4.3959545e-15 4.6651144e-15 ... 1.7571352e-15
  1.7729518e-15 2.0107218e-15]
 [7.6461978e-15 8.8348670e-15 1.0269852e-14 ... 2.3903822e-15
  1.9713190e-15 2.2685588e-15]
 [3.1927592e-14 2.6882839e-14 2.0283319e-14 ... 2.6996449e-14
  3.2472800e-14 3.4142411e-14]
 ...
 [1.3627896e-14 1.6464494e-14 2.0817635e-14 ... 2.2075251e-14
  2.2476538e-14 2.1484948e-14]
 [1.1530241e-14 1.0831453e-14 1.0261921e-14 ... 8.0661736e-15
  9.0225103e-15 9.9784030e-15]
 [6.9234216e-15 6.9943208e-15 7.1892176e-15 ... 7.0011597e-15
  7.6643022e-15 8.2142612e-15]]
[array([[0.        , 0.92437631, 0.91474378, ..., 0.86050922, 0.88162941,
        0.96265543],
       [0.92437631, 0.        , 0.97329909, ..., 0.8991307 , 0.91489142,
        0.90264869],
       [0.91474378, 0.97329909, 0.        , ..., 0.89076614, 0.90597659,
        0.90257812],
       ...,
       [0.86050922, 0.8991307 , 0.89076614, ..., 0.        , 0.99172205,
        0.91625649],
       [0.88162941, 0.91489142, 

AttributeError: 'numpy.ndarray' object has no attribute 'edges'